# Low Light to High Light Image Converter (Night Eye)
### Problem Statement:
- Predict a better lighting image from a given low/bad light image.

### Libraries

In [1]:
import numpy as np
import pandas as pd
import os
from skimage.transform import resize
import re
from tqdm import tqdm
import warnings
import rawpy
from sklearn.model_selection import train_test_split
from joblib import Parallel, delayed
warnings.filterwarnings('ignore')

In [2]:
bs = 4
img_w = 256
img_h = 256

In [3]:
dirr = r'D:\Datasets\Sony\Sony'
low, high = [], []
for file_y in os.listdir(r'D:\Datasets\Sony\Sony\long'):
    for file_x in (os.listdir(r'D:\Datasets\Sony\Sony\short')):
        if file_x.split('_')[0] == file_y.split('_')[0]:
            low.append(os.path.join(dirr, 'short', file_x))
            high.append(os.path.join(dirr, 'long', file_y))
df = pd.DataFrame(data = {'Low': low, 'High': high})
print(df.shape)
df.head()

(2697, 2)


,Low,High
0,D:\Datasets\Sony\Sony\short\00001_00_0.04s.ARW,D:\Datasets\Sony\Sony\long\00001_00_10s.ARW
1,D:\Datasets\Sony\Sony\short\00001_00_0.1s.ARW,D:\Datasets\Sony\Sony\long\00001_00_10s.ARW
2,D:\Datasets\Sony\Sony\short\00001_01_0.04s.ARW,D:\Datasets\Sony\Sony\long\00001_00_10s.ARW
3,D:\Datasets\Sony\Sony\short\00001_01_0.1s.ARW,D:\Datasets\Sony\Sony\long\00001_00_10s.ARW
4,D:\Datasets\Sony\Sony\short\00001_02_0.1s.ARW,D:\Datasets\Sony\Sony\long\00001_00_10s.ARW


In [4]:
def unpack_raw(raw):
    im = raw.raw_image_visible.astype(np.float32)
    im = np.maximum(im - 512, 0) / (16383 - 512)  # subtract the black level

    im = np.expand_dims(im, axis=2)
    img_shape = im.shape
    H = img_shape[0]
    W = img_shape[1]

    out = np.concatenate((im[0:H:2, 0:W:2, :],
                          im[0:H:2, 1:W:2, :],
                          im[1:H:2, 1:W:2, :],
                          im[1:H:2, 0:W:2, :]), axis=2)
    return out

In [5]:
def patch_extract(im, x, y, divergence = 1):
    im = im[y*divergence:y*divergence + img_w*divergence, x*divergence:x*divergence + img_w*divergence, :]
    return im

In [7]:
def raw_dump_data(idx, subset):
    low_image = rawpy.imread(df.Low.iloc[idx])
    low_image = unpack_raw(low_image)
    
    high_image = rawpy.imread(df.High.iloc[idx])
    high_image = high_image.postprocess()
    
    dirr = r'E:/Raw_Data/' + subset
    np.save('{}/Low/{}'.format(dirr, df.Low.values[idx].split('\\')[-1][:-4] + '-' + str(idx+1)), low_image, allow_pickle = True)
    np.save('{}/High/{}'.format(dirr, df.High.values[idx].split('\\')[-1][:-4] + '-' + str(idx+1)), high_image, allow_pickle = True)

In [11]:
train, val = train_test_split(df, test_size = 0.25)
train, test = train_test_split(df, test_size = 0.10)
train.shape, val.shape, test.shape

((2427, 2), (675, 2), (270, 2))

In [ ]:
res = Parallel(n_jobs = -1)(delayed(raw_dump_data)(idx, 'Train') for idx in (range(len(train))))

In [ ]:
res = Parallel(n_jobs = -1)(delayed(raw_dump_data)(idx, 'Val') for idx in (range(len(val))))

In [ ]:
res = Parallel(n_jobs = -1)(delayed(raw_dump_data)(idx, 'Test') for idx in (range(len(test))))

In [4]:
def fetch_files_loc(dirr):
    x,y=[],[]
    for dirpath, _, files in os.walk(dirr): 
        for filename in files:
            fname = os.path.join(dirpath,filename)
            if len(re.findall('Low', dirpath)) != 0:
                x.append(fname)
            else:
                y.append(fname)
    df1 = pd.DataFrame({'Low': x, 'High':y})
    return df1

In [5]:
def process(row):
    arr = np.load(row[0], allow_pickle = True)
    H = arr.shape[0]
    W = arr.shape[1]
    x = np.random.randint(0, W - img_w)
    y = np.random.randint(0, H - img_h)
    low = patch_extract(arr, x, y)

    arr = np.load(row[1], allow_pickle = True)
    H = arr.shape[0]
    W = arr.shape[1]
    high = patch_extract(arr, x, y, 2)
    
    return low, high

In [6]:
def dump_batch_data(dataframe, subset, batch_size = 32):
    low_dir = r'D:/Batch_Data/{}/Low/'.format(subset)
    high_dir = r'D:/Batch_Data/{}/High/'.format(subset)
    for idx in tqdm(range(len(dataframe))):
        X, Y = [], []
        if str(idx) + '.npy' in os.listdir(low_dir):
            pass

        else: 
            batch_idx = np.random.choice(dataframe.index.values, size = batch_size)
            rows = dataframe.iloc[batch_idx]
            for row in rows.values:
                temp = process(row)
                X.append(temp[0])
                Y.append(temp[1])
            X = np.array(X)
            Y = np.array(Y)
            np.save(low_dir + '/{}.npy'.format(idx), X)
            np.save(high_dir + '/{}.npy'.format(idx), Y)

In [7]:
train_df = fetch_files_loc(r'E:\Raw_Data\Train')
dump_batch_data(train_df, 'Train', 4)

100%|████████████████████████████████████████████████████████████████████████████| 2258/2258 [1:30:28<00:00,  2.40s/it]


In [26]:
val_df = fetch_files_loc(r'E:\Raw_Data\Val')
dump_batch_data(val_df, 'Val', 4)

100%|████████████████████████████████████████████████████████████████████████████████| 675/675 [34:50<00:00,  3.10s/it]


In [25]:
test_df = fetch_files_loc(r'E:\Raw_Data\Test')
dump_batch_data(test_df, 'Test', 4)

100%|████████████████████████████████████████████████████████████████████████████████| 270/270 [06:22<00:00,  1.42s/it]
